In [4]:
import dotenv
dotenv.load_dotenv()
import os

In [5]:
# from transformers import AutoModel
# import os

# # Load the model
# AutoModel.from_pretrained('meta-llama/Llama-2-7b-hf', 
#                                   token=os.getenv('HF_HUB_TOKEN'))

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.89s/it]


LlamaModel(
  (embed_tokens): Embedding(32000, 4096)
  (layers): ModuleList(
    (0-31): 32 x LlamaDecoderLayer(
      (self_attn): LlamaSdpaAttention(
        (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (rotary_emb): LlamaRotaryEmbedding()
      )
      (mlp): LlamaMLP(
        (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
        (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
        (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
    )
  )
  (norm): LlamaRMSNorm((4096,), eps=1e-05)
  (rotary_emb): LlamaRotaryEmbedd

In [4]:
# get the file wav_to_mel.py from https://github.com/taugastcn/SpectPrompt.git
# !curl -O https://raw.githubusercontent.com/taugastcn/SpectPrompt/main/wav_to_mel.py

In [10]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf', trust_remote_code=True, token=os.getenv('HF_HUB_TOKEN'))

In [25]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [11]:
from transformers import AutoModel, AutoTokenizer
import torch
import numpy as np
import os

model = AutoModel.from_pretrained('UniMus/OpenJMLA', trust_remote_code=True,
                                  token=os.getenv('HF_HUB_TOKEN'))
                                #   use_auth_token=True)


Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.31s/it]


In [26]:
device

device(type='cuda')

In [27]:
model.to(device)

MAEForCausalLM(
  (backbone): MAEViT(
    (patch_embed): PatchEmbed(
      (adaptive_padding): AdaptivePadding()
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (drop_after_pos): Dropout(p=0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x TransformerEncoderLayer(
        (ln1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): MultiheadAttention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0, inplace=False)
          (out_drop): DropPath()
        )
        (ln2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (ffn): FFN(
          (activate): GELU(approximate='none')
          (layers): Sequential(
            (0): Sequential(
              (0): Linear(in_features=768, out_features=3072, bias=True)
              (1): GELU

In [44]:
device = model.device
device

device(type='cuda', index=0)

In [45]:

# sample rate: 16k
music_path = 'data/borhap/segment_60_70.wav'
# 1. get logmelspectrogram
from wav_to_mel import wav_to_mel
lms = wav_to_mel(music_path)

import os
from torch.nn.utils.rnn import pad_sequence
import random



In [46]:
# get the file transforms.py from https://github.com/taugastcn/SpectPrompt.git
# !curl -O https://raw.githubusercontent.com/taugastcn/SpectPrompt/main/transforms.py

In [47]:

from transforms import Normalize, SpecRandomCrop, SpecPadding, SpecRepeat

In [48]:
transforms = [ Normalize(-4.5, 4.5), SpecRandomCrop(target_len=2992), SpecPadding(target_len=2992), SpecRepeat() ]
lms = lms.numpy()
for trans in transforms:
    lms = trans(lms)

# 2. template of input
input_dic = dict()
input_dic['filenames'] = [music_path.split('/')[-1]]
input_dic['ans_crds'] = [0]
input_dic['audio_crds'] = [0]
input_dic['attention_mask'] = torch.tensor([[1, 1, 1, 1, 1]]).to(device)
input_dic['input_ids'] = torch.tensor([[1, 694, 5777, 683, 13]]).to(device)
input_dic['spectrogram'] = torch.from_numpy(lms).unsqueeze(dim=0).to(device)
# 3. generation
model.eval()

MAEForCausalLM(
  (backbone): MAEViT(
    (patch_embed): PatchEmbed(
      (adaptive_padding): AdaptivePadding()
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (drop_after_pos): Dropout(p=0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x TransformerEncoderLayer(
        (ln1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): MultiheadAttention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0, inplace=False)
          (out_drop): DropPath()
        )
        (ln2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (ffn): FFN(
          (activate): GELU(approximate='none')
          (layers): Sequential(
            (0): Sequential(
              (0): Linear(in_features=768, out_features=3072, bias=True)
              (1): GELU

In [49]:
input_dic.keys()

dict_keys(['filenames', 'ans_crds', 'audio_crds', 'attention_mask', 'input_ids', 'spectrogram'])

In [50]:
gen_ids = model.forward_test(input_dic)
gen_text = model.neck.tokenizer.batch_decode(gen_ids.clip(0))
print(gen_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><s> This particular song falls under the genre of Hip Hop/Rap and features vocals in Chinese. The overall theme of the song is "cool" and the mood is "dynamic". The music itself is characterized by a strong beat and rhythmic flow, with a focus on the use of electronic instruments and synthesizers. The artist\'s delivery is confident and energetic, adding to the overall vibe of the song.\n\n\n\n\n\n\n\n\n\n\nore to remember\nii in Chinese culture and add a layer of depth to the song\'s meaning.\n\n\n\n\n\n\n\n own to the music, one more. This is a remix, which is a unique take on the original song by the artist 胡彦辛. The song is a great example of how music can be used to convey a message and evoke emotions in the listener.\n\n\n This is a must-listen for anyone who appreciates the artistry of hip hop and rap music.\n\n\n\n\n\n\

In [54]:
gen_text=gen_text[0]

In [55]:
# 4. Post-processing
# Given that the training data may contain biases, the generated texts might need some straightforward post-processing to ensure accuracy.
# In future versions, we will enhance the quality of the data.
gen_text = gen_text.split('<s>')[-1].split('\n')[0].strip()
gen_text = gen_text.replace(' in Chinese','')
gen_text = gen_text.replace(' Chinese','')
print(gen_text)


This particular song falls under the genre of Hip Hop/Rap and features vocals. The overall theme of the song is "cool" and the mood is "dynamic". The music itself is characterized by a strong beat and rhythmic flow, with a focus on the use of electronic instruments and synthesizers. The artist's delivery is confident and energetic, adding to the overall vibe of the song.
